# 모델 성능 비교
- 유사 판례 검색 시스템에 사용될 두 모델의 성능 비교
1. FastText
    - ngram 기반 학습 모델
    - 학습 데이터 : 판례 데이터
    
2. BERT
    - Transformer 기반 모델
    - 학습 데이터 : 카카오브레인 - STS 데이터

## 1. 패키지 로드

In [30]:
import pandas as pd
import numpy as np
import pickle as pk
import re
import tensorflow as tf

from gensim.models import FastText
from sentence_transformers import SentenceTransformer
from sqlalchemy import create_engine
from nltk import sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity

- 판례 데이터 로드

In [19]:
host = 'localhost'
user = 'root'
password = '1234'
db = 'cases'
port = 3306

In [20]:
db_connection_str = f'mysql+pymysql://{user}:{password}@{host}:{port}/{db}'
db_connection = create_engine(db_connection_str)
connect = db_connection.connect()

In [21]:
df = pd.read_sql("SELECT * FROM case_info", connect)
df.head(3)

,판례일련번호,사건명,사건번호,선고일자,선고,법원명,법원종류코드,사건종류명,사건종류코드,판결유형,판시사항,판결요지,참조조문,참조판례,판례내용
0,64437,사해행위취소등,2003나1746,2003-07-10,선고,대전고법,4002,민사,4001,판결: 상고,파산자의 채권에 기한 사해행위취소의 소에서 사해행위 사실을 알았는지 여부를 파산관재...,파산법이 파산관재인에게 파산재단에 관한 소에 있어 원고 또는 피고가 된다고 한 것은...,"민법 제406조 제2항\n,\n\n 파산법 제7조","\n대법원 1990. 11. 13. 선고 88다카26987 판결(공1991, 57)","【원고,항소인】 파산자 주식회사 충남상호신용금고의 파산관재인 예금보험공사 (소송..."
1,64438,위증,2008도3300,2008-06-26,선고,대법원,4002,형사,4001,판결,[1] 공범인 공동피고인이 다른 공동피고인에 대한 공소사실에 관하여 증인적격이 있는...,[1] 공범인 공동피고인은 당해 소송절차에서는 피고인의 지위에 있으므로 다른 공동피...,"[1] \n \n 형사소송법 제146조, \n ...",[1] \n \n 대법원 1983. 10. 25. 선고 8...,【피 고 인】 【상 고 인】 검사【원심판결】 수원지법 2008. 4. 1. 선고 2...
2,64439,상고장각하명령에대한이의,2007무77,2008-06-02,자,대법원,4002,일반행정,4001,결정,[1] 소송구조신청에 대한 기각결정이 확정되기 전에 소장 등에 인지를 첩부하지 않았...,[1] \n \n 행정소송법 제8조 제2항에 의하여...,"[1] \n \n 민사소송 등 인지법 제1조, \n ...",[1] \n \n 대법원 1993. 1. 25.자 92마1...,【재항고인(선정당사자)】 【상 대 방】 인천광역시장【원심명령】 서울고법 2007. ...


* 사용자 입력

In [6]:
user_input = "사건 내용은 1년넘게 사귄 남자친구가 있었습니다 그 남자친구가 바람피고 저한테 걸려서 헤어지는과정에 말다툼을했었는데, 저한테 마주치면 코뼈부쉰다,팬다 어쩐다 협박을 했었어요. 몇일뒤 지나가는길에 그 남자친구를 마주쳤습니다 남자친구와 그의 친구들 총 5명이 있었는데 저한테 얘기를 하자고하길래 친구들있으니 저쪽가서 이야기하자 라고 하고 자리를 옮겼어요. (옮겼지만 그 친구들은 다시 남자친구와 제가 있는쪽으로 와서 구경했고요) 자리 옮기고 나서 얘기를 하려고 하니 얘기는커녕 그때 무슨생각으로 자기한테 욕을했냐 라면서 따지기 시작했고 일방적으로 욕을 먹기 시작했습니다. 얘기중에 손으로 제 팔 내려치길래 저는 그애가 분노조절도 못하고 충동적인 행동을 하는것을 잘 알기에 진정해라,이래서 너한테 좋을거없다 라고 수차례 얘기했음에도 자기 화를 이기지못해 저를 폭행했습니다. 머리를 손으로 내려치고 발로 팔을 차서 핸드폰 날라가서 망가지고, 제가 걔를 쳐다보니 뭘보냐면서 안면상그대로 손으로 맞았습니다 그러다 같이있던 제 친구가 112에 신고를하고 경찰분들 오기를 기다렸어요 그러다 사건접수하고 전 119타고 대학병원응급실가서 검사와 약처방받고 집으로 갔습니다. 경찰서까지 넘어가서 조사받고 단순 폭행으로 검찰 넘어가는중이에요"

## 2. FastText 성능 테스트

### 1) 모델 로드

In [2]:
fasttext = FastText.load('./fasttext/fasttext.model')

### 2) 임베딩 값 로드

In [5]:
with open("./data/fasttext_embedding.pickle", 'rb') as fr:
    fasttext_embedding = pk.load(fr)

### 3) 사용자 입력 전처리

In [9]:
clean_user_input = re.sub("[^가-힣. ]", '', user_input)
clean_user_input

'사건 내용은 년넘게 사귄 남자친구가 있었습니다 그 남자친구가 바람피고 저한테 걸려서 헤어지는과정에 말다툼을했었는데 저한테 마주치면 코뼈부쉰다팬다 어쩐다 협박을 했었어요. 몇일뒤 지나가는길에 그 남자친구를 마주쳤습니다 남자친구와 그의 친구들 총 명이 있었는데 저한테 얘기를 하자고하길래 친구들있으니 저쪽가서 이야기하자 라고 하고 자리를 옮겼어요. 옮겼지만 그 친구들은 다시 남자친구와 제가 있는쪽으로 와서 구경했고요 자리 옮기고 나서 얘기를 하려고 하니 얘기는커녕 그때 무슨생각으로 자기한테 욕을했냐 라면서 따지기 시작했고 일방적으로 욕을 먹기 시작했습니다. 얘기중에 손으로 제 팔 내려치길래 저는 그애가 분노조절도 못하고 충동적인 행동을 하는것을 잘 알기에 진정해라이래서 너한테 좋을거없다 라고 수차례 얘기했음에도 자기 화를 이기지못해 저를 폭행했습니다. 머리를 손으로 내려치고 발로 팔을 차서 핸드폰 날라가서 망가지고 제가 걔를 쳐다보니 뭘보냐면서 안면상그대로 손으로 맞았습니다 그러다 같이있던 제 친구가 에 신고를하고 경찰분들 오기를 기다렸어요 그러다 사건접수하고 전 타고 대학병원응급실가서 검사와 약처방받고 집으로 갔습니다. 경찰서까지 넘어가서 조사받고 단순 폭행으로 검찰 넘어가는중이에요'

### 4) 사용자 입력 문장 토큰화

In [10]:
user_sentence_tokened = sent_tokenize(clean_user_input)
user_sentence_tokened

['사건 내용은 년넘게 사귄 남자친구가 있었습니다 그 남자친구가 바람피고 저한테 걸려서 헤어지는과정에 말다툼을했었는데 저한테 마주치면 코뼈부쉰다팬다 어쩐다 협박을 했었어요.',
 '몇일뒤 지나가는길에 그 남자친구를 마주쳤습니다 남자친구와 그의 친구들 총 명이 있었는데 저한테 얘기를 하자고하길래 친구들있으니 저쪽가서 이야기하자 라고 하고 자리를 옮겼어요.',
 '옮겼지만 그 친구들은 다시 남자친구와 제가 있는쪽으로 와서 구경했고요 자리 옮기고 나서 얘기를 하려고 하니 얘기는커녕 그때 무슨생각으로 자기한테 욕을했냐 라면서 따지기 시작했고 일방적으로 욕을 먹기 시작했습니다.',
 '얘기중에 손으로 제 팔 내려치길래 저는 그애가 분노조절도 못하고 충동적인 행동을 하는것을 잘 알기에 진정해라이래서 너한테 좋을거없다 라고 수차례 얘기했음에도 자기 화를 이기지못해 저를 폭행했습니다.',
 '머리를 손으로 내려치고 발로 팔을 차서 핸드폰 날라가서 망가지고 제가 걔를 쳐다보니 뭘보냐면서 안면상그대로 손으로 맞았습니다 그러다 같이있던 제 친구가 에 신고를하고 경찰분들 오기를 기다렸어요 그러다 사건접수하고 전 타고 대학병원응급실가서 검사와 약처방받고 집으로 갔습니다.',
 '경찰서까지 넘어가서 조사받고 단순 폭행으로 검찰 넘어가는중이에요']

### 5) 사용자 입력 임베딩

In [12]:
embedding = None
count = 0
for sentence in user_sentence_tokened:
    if embedding is None:
        embedding = fasttext.wv.get_sentence_vector(sentence)
    else:
        embedding += fasttext.wv.get_sentence_vector(sentence)
    count += 1

f_embedding = embedding / count

### 6) 코사인 유사도 계산

In [18]:
f_similarity = cosine_similarity([f_embedding], fasttext_embedding)
f_similarity = list(enumerate(f_similarity[0]))
f_similarity = sorted(f_similarity, key=lambda x : x[1], reverse = True)

### 7) 유사 판례 (상위 5개) 확인

In [26]:
precidents = []

for i in range(5):
    idx = f_similarity[i][0]
    sim = round(f_similarity[i][1] * 100, 2)
    
    precidents.append([df['판례일련번호'][idx], df['사건명'][idx], df['사건종류명'][idx], df['판례내용'][idx], sim])

In [27]:
f_result = pd.DataFrame(precidents, columns=['판례일련번호', '사건명', '사건종류명', '판례내용', '유사도'])
f_result

,판례일련번호,사건명,사건종류명,판례내용,유사도
0,71041,살인(베트남신부 살인사건),형사,【피 고 인】 【항 소 인】 피고인 및 검사【검 사】 서홍기【변 호 인】 변호...,94.90
1,189452,폭력행위등처벌에관한법률위반,형사,【피 고 인】 【상 고 인】 검 사【원심판결】 광주지방법원 1991.1.17....,94.79
2,123173,"강도상해,폭력행위등처벌에관한법률위반",형사,【피 고 인】 【상 고 인】 피고인【변 호 인】 변호사 오행남【원심판결】 ...,94.73
3,125411,업무상과실치상,형사,【피 고 인】 【상 고 인】 피고인【원심판결】 춘천지방법원 1988.12.1. ...,94.52
4,129749,정정보도,민사,【원 고】 【피 고】 【제2심판결】 \n서울고법 1997. 4. 16. ...,94.49


## 3. SBERT 성능 테스트

### 1) 모델 로드

In [32]:
with tf.device("/device:GPU:0"):
    sbert = SentenceTransformer('jhgan/ko-sbert-sts')

### 2) 임베딩 값 로드

In [33]:
embedding_df = pd.read_sql("SELECT * FROM case_embedding", connect)

In [34]:
merge_df = pd.merge(left=df, right=embedding_df, how='inner', on='판례일련번호')

In [36]:
merge_df['임베딩'] = merge_df['임베딩'].apply(lambda x : np.frombuffer(x, dtype=np.float32))

### 3) 사용자 입력 임베딩
- 앞서 전처리한 입력 그대로 사용

In [38]:
embedding = None
count = 0
for sentence in user_sentence_tokened:
    if embedding is None:
        embedding = sbert.encode(sentence)
    else:
        embedding += sbert.encode(sentence)
    count += 1

s_embedding = embedding / count

### 4) 코사인 유사도 계산

In [41]:
s_similarity = cosine_similarity([s_embedding], merge_df['임베딩'].to_list())
s_similarity = list(enumerate(s_similarity[0]))
s_similarity = sorted(s_similarity, key=lambda x : x[1], reverse = True)

### 5) 유사 판례 (상위 5개) 확인

In [44]:
precidents = []

for i in range(5):
    idx = s_similarity[i][0]
    sim = round(s_similarity[i][1] * 100, 2)
    
    precidents.append([df['판례일련번호'][idx], df['사건명'][idx], df['사건종류명'][idx], df['판례내용'][idx], sim])

In [45]:
s_result = pd.DataFrame(precidents, columns=['판례일련번호', '사건명', '사건종류명', '판례내용', '유사도'])
s_result

,판례일련번호,사건명,사건종류명,판례내용,유사도
0,76370,강간치상피고사건,형사,【피 고 인】 【주 문】 피고인은 무죄【이 유】 이 사건 공소사실은 ...,58.51
1,118497,강간치사피고사건,형사,"【피 고 인】 【주 문】피고인은 무죄【이 유】이 사건 공소사실의 요지는,...",58.04
2,101984,무고,형사,【피 고 인】 【상 고 인】 검사【원심판결】 \n서울형사지방법원 1985.11....,57.69
3,92930,폭행치사,형사,【피 고 인】 【상 고 인】 검사【원 판 결】 \n서울고등법원 1975.10.1...,56.23
4,227633,폭력행위등처벌에관한법률위반,형사,【피 고 인】 피고인 1 외 1인 【주 문】 피고인들은 각 무죄...,56.15


## 4. FastText와 SBERT 결과 비교

### 1) FastText 결과 분석

#### (1) 유사도 1순위 판례내용

In [46]:
f_result['판례내용'][0]

'【피 고 인】 【항 소 인】 피고인 및 검사【검    사】 서홍기【변 호 인】 변호사 최권주【제1심판결】 대전지법 천안지원 2007. 10. 9. 선고 2007고합118 판결【주    문】피고인 및 검사의 항소를 모두 기각한다.【이    유】  1. 항소이유의 요지  피고인(양형과중), 검사(양형과경)   2. 판 단  가. 양형의 조건  ○ 전과  음주 중의 폭력행위 등으로 인하여 6회의 벌금형 처벌전력이 있다.   ○ 성장배경 및 생활환경(판결 전 조사결과 및 당심 감정인 박상규의 감정결과)  피고인은 경남 창녕군에서 3남 중 막내로 태어나 아버지의 가출 후 홀어머니 슬하에서 가난하게 생활하면서 초등학교와 중학교를 고향에서 졸업하였으나, 경제적 빈곤과 부모의 무관심으로 고등학교에 진학하지 못하였다. 중학교를 졸업하고 1년 가량 집안 농사일을 돕다가 마산시 등에서 몇 년간 공원생활을 하였다. 군복무를 마치고 상경하여 자동차학원 강사, 노동 등을 하면서 방송통신고등학교를 마쳤다. 피고인은 41살 이후에서야 본격적으로 결혼하고자 하였고, 피고인은 그 이유에 관하여, “나이 먹은 남자가 혼자 있으니 부끄럽고 남들이 병신같이 볼 것이라고 생각하여 결혼하기로 마음먹었다”고 진술하고 있다. 우리나라 여성들과 몇 번 선을 보았으나 성사되지 못하였다. 피고인은 “여자들이 너무 현실적인 것을 따진다”고 토로하였다. 천안에서 노동일을 하면서 우연히 생활정보지를 보고 국제결혼정보업체를 알게 되었으며, 전재산에 가까운 1,000만 원을 지급하고 국제결혼정보업체를 통하여 2006. 12. 23. 베트남에서 피해자와 결혼식을 올린 후 2007. 5. 16.부터 천안에서 피해자와 결혼생활을 시작하였다.  ○ 범행 전후의 심리상태(당심 감정인 박상규의 피고인에 대한 정신감정결과)  피고인은 알코올 문제가 있어 보인다. 알코올 섭취 후에 공격성 조절이 잘 되지 않아 타인에 대하여 폭력적 행위를 하는 등 대인관계의 문제가 있었던 것으로 보인다. 피고인은 피해망상적 사고경향과 외상 후 스트

- 내용요약 : 폭행, 가정폭력, 우발성 폭행, 살인

#### (2) 유사도 2순위 판례 내용

In [47]:
f_result['판례내용'][1]

'【피 고 인】 【상 고 인】   검  사【원심판결】 광주지방법원 1991.1.17. 선고 90노620 판결【주    문】  원심판결을 파기하고 사건을 광주지방법원 합의부에 환송한다.【이    유】  상고이유에  대하여  1. 원심판결 이유에 의하면, 원심은, ‘피고인이 1989.5.8. 23:35경 광주 동구 B 소재 C약국앞길에서 피해자 D의 오른손을 잡고 비틀어 그에게 전치 약 2주의 우측 제4, 5중수지절 관절염좌상을 가하였다’는 공소사실에 대하여, 설시 증거로, “피고인이 1989.5.8. 23:35경 E와 피해자 D를 자기 택시에 태우고 광주 동구 B 소재 C약국 앞길에 도착하여 그곳 시장골목으로 들어가자는 피해자의 요청을 거절하자, 피해자 등이 항의하여 서로 언쟁하는 과정에서, E가 손가방과 하이힐로 피고인의 머리를 1회씩 때리고 어깨를 1회 물 때, 피해자도 가세하여 피고인의 머리를 1회 물어 피고인에게 전치 약 2주의 두피부열상, 우측상지교상 등을 가한 후 도주하므로, 피고인이 도주하는 피해자의 손을 붙잡고 관할 파출소로 데리고 가기 위하여 잡아끄는 과정에서 동인이 전치 약 2주의 제 4, 5중수지절 관절염좌상을 입게 된 사실을 인정한 다음, 피고인이 위 상해행위에 이르게 된 경위와 그 목적수단 및 의사 등 제반 사정에 비추어 보면 이러한 행위는 사회통념상 허용될만한 정도의 상당성이 있는 것으로서 위법성이 결여되어 \n        형법 제 20조의 정당행위에 해당한다”고 판시하였다.\n        결국 피해자 등이 피고인을 구타한 이유는, 원심의 설시에 의하면 오로지 피고인이 시장골목 안으로 택시를 운행하라는 피해자 등의 요구를 거절한 게 발단이 되어 서로 말다툼 끝에 감정이 격해졌기 때문이라고 풀이되고, 한편 피고인이 피해상황에 대하여 원심이 채택한 F의 경찰 진술은 “E가 택시안에서 운전사에게 욕을 하면서 내리라고 하여 운전사가 내리니까 함께 내린 후 핸드백으로 운전사의 머리를 때렸고 핸드백이 땅에 떨어져 버리자 하이힐을 벗어 뒷굽으로 

- 내용요약 : 폭행, 우발적 폭행, 욕설

#### (3) 유사도 3순위 판례 내용

In [48]:
f_result['판례내용'][2]

'【피 고 인】 【상 고 인】   피고인【변 호 인】   변호사  오행남【원심판결】 \n서울고등법원 1993.2.16. 선고 92노4443 판결【주    문】  원심판결을 파기하고 사건을 서울고등법원에 환송한다.【이    유】  피고인과 변호인의 상고이유에 대하여  이 사건 공소사실에 의하면, 피고인은 1992.4.21. 21:30경 인천 북구 효성동 395의 95 소재 코리우드 주식회사 앞길에서 그 곳을 지나가던 피해자 \n        1(여, 35세)에게 접근하여 팔로 목을 감아 쓰러뜨리고 “소리치면 죽이겠다. 돈 있는 거 다 내놔라”라고 위협하면서 계속 목을 졸라 반항을 억압한 다음 금품을 강취하려 하였으나 피해자\n         1이 “강도야”라고 소리지르면서 도망하여 그 뜻을 이루지 못하고 미수에 그치고 이로 인하여 \n          피해자 1에게 약 10일간의 치료를 요하는 전경부좌상 등의 상해를 가하고, 같은 날 21:40경 인천 북구 청천2동 172의 12 소재 신창튜빙 주식회사 앞길에서 피해자 \n        2(남, 33세)\n        가 피고인에게 경찰서로 갈 것을 요구하면서 혁대를 붙잡았다는 이유로 그의 손가락을 꺾어 \n          피해자 2에게 약 10일간의 치료를 요하는 좌측수배좌상을 가한 것이라고 되어 있다. 제1심 판결은 제1회 공판조서 중 피고인의 일부진술기재, 제2회 공판조서 중 증인 \n        피해자 2의 진술기재 및 제3회 공판조서 중 증인 \n        피해자 1의 진술기재, 증인 송영운의 법정진술, 의사의 \n        피해자 1, \n        2에 대한 진단서를 종합하여 피고인에 대하여 유죄판결을 선고하였고, 원심은 이를 지지하여 피고인의 항소를 기각하였다.\n        그러나 \n        제1심 판결이 피고인에 대한 강도상해의 범죄사실을 인정함에 있어서 채용한 증거를 검토해 볼 때 피고인에 대하여 유죄를 단정할 증거가 될 수 있는 것인지 의심을 떨쳐 버릴 수 없다.  

- 내용요약 : 강도, 폭행, 강도상해

#### (4) 유사도 4순위 판례 내용

In [49]:
f_result['판례내용'][3]

'【피 고 인】 【상 고 인】   피고인【원심판결】 춘천지방법원 1988.12.1. 선고 88노299 판결【주    문】  원심판결을 파기하고, 사건을 춘천지방법원 합의부에 환송한다.【이    유】  피고인의 상고이유를 본다.  1. 원심판결 이유에 의하면, 원심은 피고인이 1987.6월경 강원 고성군 토성면 아야진 6리 소재 아야진 조선소에서 도목수로서 선박건조 및 수리공사에 대한 총괄적인 감독과 작업지시업무에 종사하던 중 1987.6.3. 오전에는 위 공장기관실에서 기관실 앞에 설치되어 있는 레일을 이용하여 뭍으로 선박을 끌어올리는 작업을 지시 감독하였는 바, 그 작업은 20마력의 원동기 활에 피대를 감아 로울러에 연결시키고, 직경 약 1.5센치미터의 쇠줄을 배 받침대(선대라고 함)에 매달고 바다에 정박한 배를 선대위에 올려놓은 다음, 쇠줄을 로울러에 감고 로울러의 안전레버를 작동위치에 두어 원동기의 힘으로 로울러를 회전시켜 레일 위에 선박을 끌어올리는 작업으로서 위 안전레버는 원동기가 작동하는 중에도 안전쪽으로 밀면 로울러가 정지하도록 되어 있는 사실, 사고당일 피고인과 목공인 박의곤 및 피해자 등은 무게 7톤의 정진호를 비롯한 선박 3척을 뭍으로 끌어올리는 작업을 하였는데, 위 박 의곤은 엔진실에서 밖에 있는 피고인의 신호에 따라 원동기를 작동시키고 피해자는 피고인과 함께 밖에서 선박을 레일 위에 고정시키는 작업을 하여 일단 위 정진호를 마지막으로 선박 3척을 모두 끌어올렸던 사실, 그런데 피고인은 추가로 배 1척을 더 끌어올리기 위하여 이미 기관실에서 약 15 내지 20미터 앞까지 끌어올린 위 정진호를 조금 더 끌어올릴 필요가 생겨 피해자에게 배를 좀더 올려야겠다고 이야기를 하자 피해자는 기관실로 갔던바, 당시 위 박의곤은 배를 끌어올리는 작업이 모두 끝난 것으로 알고 원동기를 가동한 상태에서 로울러의 레버장치만 안전에 두어로울러를 정지시켜둔 채 화장실에 가고 기관실에 없었던 사실, 그러자 피해자는 혼자서 쇠줄을 로울러에 감고 위 레버를 풀어 로울러를

- 내용요약 : 안전사고, 관리소홀

#### (5) 유사도 5순위 판례 내용

In [50]:
f_result['판례내용'][4]

'【원    고】 【피    고】 【제2심판결】 \n서울고법 1997. 4. 16. 선고 97나47141 판결【주    문】  1. 원고의 청구를 모두 기각한다.  2. 소송비용은 원고의 부담으로 한다.【청구취지】  피고들은 연대하여 원고에게 금 200,000,000원 및 이에 대하여 이 사건 소장 부본 송달 익일부터 완제일까지 연 2할 5푼의 비율에 의한 금원을 지급하라. \n        피고 1 주식회사는 이 판결을 송달받은 후 최초로 방송하는 편집이 완료되지 않은 오후 9시 MBC 뉴스데스크 카메라출동 시간에 30초간 화면상의 문자와 대사로 별지 제1목록 기재 정정보도문을 방송하라는 판결\n      【이    유】  1. 기초사실  다음의 각 사실은 당사자 사이에 다툼이 없거나, 갑 제1호증의 11, 12, 14, 15, 16, 17, 18, 을 제1호증, 을 제2호증의 3, 4, 5의 각 기재 및 당원의 검증 결과에 변론의 전취지를 종합하여 보면 이를 각 인정할 수 있고, 이에 반하는 갑 제1호증의 5, 6, 7, 13, 19의 각 기재는 이를 각 믿지 아니하고 달리 반증 없다.  (1) 원고는 소외 주식회사 서울방송(이하 서울방송이라 한다)의 기자로서 KBS 및 SBS의 주말뉴스 앵커를 거쳐 현재 위 서울방송\n        에(이하 생략) 근무하는 사람이고, \n        피고 1 주식회사는 \'MBC\'라는 텔레비전 및 라디오 방송국을 운영하는 회사로서 매일 저녁 9시 \'MBC 뉴스데스크\'라는 뉴스 프로그램을 제작·방송하고 있고, \n        피고 2는 \n        피고 1 주식회사의 기획취재부 부장, \n        피고 3은 같은 부 차장, \n        피고 4는 같은 부 기자로 근무하는 사람들로서 위 \'MBC 뉴스데스크\'에서 방영되는 \'카메라출동\'이라는 프로그램의 제작을 담당하고 있다.\n        (2) \n        피고 2는 1996. 10. 14.과 같은 달 15. 위 \'카메라출동\'에서 방

- 내용요약 : 오보(잘못된보도), 정정요청, 명예훼손, 음주운전

**내용종합** : 폭행관련 3, 안전사고 1, 오보 1

### 2) SBERT 결과 분석

#### (1) 유사도 1순위 판례내용

In [51]:
s_result['판례내용'][0]

'【피 고 인】 【주    문】  피고인은 무죄【이    유】  이 사건 공소사실은 피고인은 1985.9.30. 19:10경 서울 강남구 \n        (상세주소 생략)에 있는 피고인이 경영하는 목공소의 지하실 방에서, 피고인에게 남동생의 취직을 부탁하러 위 목공소에 찾아온 \n        피해자(여, 27세)를 보고 갑자기 욕정을 일으켜 "소리를 지르면 너는 유부녀이고 나는 총각이니 누가 망신이냐"라고 말하여 그녀의 수치심을 이용하여 반항을 못하게 한 다음 옷을 벗기고 강제로 눕혀 허벅지를 짓누르면서 그녀를 1회 간음하여 강간하고 그로 인하여 그녀로 하여금 약 4일간의 치료를 요하는 양측 대퇴하단 전면 타박상을 입게 한 것이다라고 함에 있고, 피고인도 이 법정에서 위와 같은 사실관계는 이를 시인하고 검사가 제출한 모든 증거자료를 이 사건의 증거로 함에 동의하고 있다.\n        그런데, 강간죄에 있어서의 폭행 또는 협박은 피해자에게 단지 외포심이나 수치심을 야기할 정도로서는 피해자의 반항을 현저히 곤란하게 하거나 억압할 정도의 유형력을 행사하던지 해악을 고지함을 요한다고 할 것인바, 위 공소사실과 같이 피고인이 피해자에게 "소리를 지르면 너는 유부녀이고 나는 총각이니 누가 망신이냐"는 말을 하여 피해자에게 수치심을 느끼게 한 것이 과연 강간죄에 있어서 요구되는 폭행이나 협박에 해당하는 것인지에 관하여 살피건대, 피고인이 이 법정에서 한 진술에 검사 및 사법경찰관 사무취급이 작성한 피고인에 대한 각 피의자신문조서와 검사 및 사법경찰리가 작성한 \n        피해자에 대한 각 진술조서의 각 진술기재와 \n        피해자가 작성한 고소장, 고소취소장, 합의서 및 의사 \n        공소외인이 작성한 \n        피해자에 대한 상대진단서의 각 기재를 종합하여 보면, 피해자는 이 사건 간음행위가 있었던 위 목공소 건물의 1층에 2년전부터 세들어 살고 있는 가정주부로서, 금년 봄부터 위 건물 지하실에서 소파제작업을 시작한 피고인과는 이 사건

- 내용요약 : 강간, 폭행, 협박, 합의금

#### (2) 유사도 2순위 판례내용

In [52]:
s_result['판례내용'][1]

'【피 고 인】 【주    문】피고인은 무죄【이    유】이 사건 공소사실의 요지는, 피고인은 1988. 8. 4. 13:30경 제주 남제주군 표선면 \n        (상세주소 생략) 소재 피해자 \n        공소외 1(15세)의 방에서 피해자가 방안에서 바지를 벗고 옷을 갈아 입는 것을 보고 욕정을 일으켜 강간할 목적으로 그녀가 있는 방으로 가 양손으로 껴안으려고 하다가 피해자가 소리를 지르며 창문을 뛰어 넘어 도망가는 바람에 그 목적을 이루지 못하고 미수에 그쳤으나 도망가는 피해자를 붙잡으려고 하다가 동인을 창문 밖으로 넘어뜨려 동인에게 약 7일간의 치료를 요하는 좌우측슬관절부위타박상등을 입게한 것이다라고 함에 있으므로 살피건대, 강간치상죄라 함은 강간죄를 범하여 부녀를 상해에 이르게 한 죄로서 그 전제로 되는 강간죄는 기수에 이를 것을 요하지 아니하고 미수라도 족하나 예비의 단계에서 나아가 실행의 착수가 있음을 요하므로 적어도 부녀를 간음하기 위하여 폭행, 협박이 개시되었음을 요한다 할 것이고 이 때의 폭행, 협박은 상대방의 의사를 억압할 정도에 이를 것을 요하지 아니하나 적어도 상대방의 반항을 현저히 곤란하게 할 정도에 이르러야 하며 이러한 판단에는 상대방의 연령, 건강상태, 경력 등의 주관적 사정과 범행이 행하여진 시간, 장소 등의 주변환경과 같은 객관적 사정이 구체적으로 고려되어야 할 것인 바, 먼저 이 점에 관하여 보건대 피고인은 경찰이래 이 법정에 이르기까지 일관하여 도대체 이 사건 범행 장소에 간일 조차 없다고 범행일체를 극구부인하고 있고 검사가 이 사건 공소사실을 유죄로 하기 위하여 제출한 증거 중 이 점에 관한 적법한 증거로는 증인 \n        공소외 1, \n        2의 이 법정에서의 진술, 검사 및 사법경찰관사무취급 작성의 \n        공소외 1에 대한 진술조서의 기재, 사법경찰관사무취급 작성의 \n        공소외 2에 대한 진술조서의 기재, 사법경찰관사무취급 작성의 검증조서 중 \n        공소외 1

- 내용요약 : 강간, 폭행

#### (3) 유사도 3순위 판례내용

In [53]:
s_result['판례내용'][2]

'【피 고 인】 【상 고 인】   검사【원심판결】 \n서울형사지방법원 1985.11.19 선고 85노4104 판결【주    문】  상고를 기각한다.【이    유】  검사의 상고이유를 본다.  원심판결 이유에 의하면, 원심은 그 거시증거에 의하여 피고인과 공소외\n        인이 이 사건 주주총회 회의장 앞뒤쪽에서 상대방을 비난하다가 감정이 격해져서 서로 의자를 밀치면서 달려나와 손으로 상대방의 멱살을 붙잡고 밀고 당기면서 회의장출구쪽으로 나가던중 피고인이 넘어지면서 의자에 왼쪽다리를 부딪쳐서 상처를 입게 된 사실을 인정한 다음, 이와 같이 서로 멱살을 잡고 밀고 당기는 과정에서 상해를 입게 된 이상 피고인이 \n        공소외인으로부터 폭행당하여 상해를 입었다고 고소한 것이 허위사실을 신고한 것이라고 볼 수는 없다하여 무죄를 선고한 제1심 판결을 그대로 유지하고 있는바, 원심이 취사선택한 증거관계를 기록에 비추어 검토하면 원심의 위와 같은 사실인정과 판단은 정당하게 수긍이 가고 거기에 채증법칙을 어기거나 무고죄의 법리를 오해한 위법이 있다할 수 없으므로 논지는 결국 이유없다.\n        따라서 상고를 기각하기로 하여 관여법관의 일치 된 의견으로 주문과 같이 판결한다.대법관 윤일영(재판장) 이명희 최재호 황선당 '

- 내용요약 : 폭행

#### (4) 유사도 4순위 판례내용

In [54]:
s_result['판례내용'][3]

'【피 고 인】 【상 고 인】   검사【원 판 결】 \n서울고등법원 1975.10.17. 선고, 74노302 판결【주    문】  상고를 기각한다.【이    유】  검사의 상고이유를 판단한다.원판결 판단에 의하면 원심은 피고인은 술이 거나해서 지나가다가 술에 만취된 피해자에게 어깨침을 당하며 명함 주기를 요구받고 거절하니 피해자가 떠밀어 빙판길에 넘어졌는데 피해자는 피고인의 외투깃을 잡아당겨 일으키려고 하므로 그의 행패에서 빠져 나가려고 외투깃을 잡고 있는 피해자의 오른손을 뿌리쳤는데 이로 인하여 그곳이 빙판길인데다가 술에 만취된 탓으로 몸을 제대로 가누지 못하고 피해자는 넘어져서 머리를 땅에 부딛쳐 설시 상해를 입고 설시 일자에 죽었다는 사실을 인정하고 사정이 이상과 같은 경우라면 자기 외투깃을 잡힌 피해자의 오른손을 뿌리친 행위는 그 행위에 이른 과정과 목적수단 및 행위자의 의사등 제반사정에 비추어 사회통념상 허용될 상당성이 있다고 할 것이어서 위법성이 결여된 행위라고 판단하고 있는데 위 인정판단은 옳게 시인되며 폭력의 법리의 오해나 채증법칙을 위배한 사실오인의 위법이 있다고 단정하기 어렵다.\n        논지는 원판결과 견해를 달리하고 원판결을 비의하는 것으로서 채용할 길이 없다. 그러므로 일치된 의견으로 주문과 같이 판결한다.대법관 김윤행(재판장) 이영섭 민문기 김용철 '

- 내용요약 : 상해

#### (5) 유사도 5순위 판례내용

In [55]:
s_result['판례내용'][4]

'【피 고 인】 피고인 1 외 1인        【주    문】  피고인들은 각 무죄.【이    유】  1. 이 사건 공소사실의 요지  피고인들은 공동하여, 1992.8.5. 17:00경 충북 청원군 문의면 염티리 소재 염티다리 밑에서 술을 마시고 놀던 중 일행인             피해자(남, 43세)가 "나도 공사장 인부의 팀장을 할 수 있다"라고 말하자 이에 화가 나서             피고인 1이 "저 새끼 죽여"라고 소리치며 손바닥과 주먹으로 피해자의 목부분과 등부분을 수회 때리고,             피고인 2는 "수도란 그렇게 하는 것이 아니다"라며 손바닥 끝으로 피해자의 목부분과 어깨부분을 수회 때려 피해자에게 약 5주간의 치료를 요하는 경추부외상성디스크 등의 상해를 가하였다.        2. 피고인들의 주장과 이 사건의 쟁점  가.             피고인 1은, 경찰 이래 이 법정에 이르기까지 자신은 공소사실 기재 일자(음력 7월 7석임)에 자신이 데리고 일을 하는 콘크리트 할석공 10여 명(            상피고인 2,             피해자 포함)과 함께 개를 1마리 잡아서 야유회차 공소사실 기재 장소에 간 바는 있으나, 같은 날 12:00경 현장에 도착하여 개고기를 안주삼아 소주를 마시며 놀던 중             피해자는 음주 만취되어 자신의 일행이나 같은 장소에 놀러 왔던 ○○○○ 광원들에게 시비를 자청하다가 아무도 응대하지 않자 혼자 개천변에 누워서 잠을 잤고,             피고인 1과             피해자를 제외한             상피고인 2 등 나머지 일행은 같은 날 14:00부터 15:00까지 사이에 삼삼오오 흩어져 집으로 돌아갔으며, 자신은 위 ○○○○ 광원들과 술을 더 마시고는 술에 취하여 그 곳 다리 위에서 잠을 자다가 같은 날 16:20경 깨어나 위 ○○○○ 광원들인 공소외 1, 공소외 2 등과 함께 시내버스를 타고 청주로 돌아왔고, 돌아오기 직전 술에 취하여 자고 있던 

- 내용요약 : 폭행, 욕설, 상해

**내용종합** : 폭행관련 5

## 5. 결론
> 폭행, 상해관련 사용자의 사건에 대해서 상위 5개의 결과 중 fastText 모델은 60% 비슷한 판례를 SBERT는 100% 비슷한 판례 검색하였다. 이로비추어 보았을때, 비록 SBERT가 법관련 데이터로 학습을 하지 않았더라도 문맥상의 유사도를 잘 찾아내어 높은 정확도를 보이는 것을 알 수 있다.